In [43]:
import folium
import mf2py
import json

### a. Obtener y visualizar coordenadas geograficas

In [36]:
def get_geo_data(name):
    """
        Devuelve los datos de geolocalizacion correspondiente
        a una ciudad registrada en Wikipedia.
    :param name: Nombre de la ciudad
    :return: Tupla con latitud y longitud
    """
    name = name.replace(" ", "_")
    url = "https://es.wikipedia.org/wiki/" + name
    x = mf2py.Parser(url = url)
    x = x.to_dict(filter_by_type='h-card')[0]['properties']['geo'][0]
    x = x['value'].split(",")
    return x

In [27]:
ciudades = ["Trelew", "Rawson", "Munich", "Nueva York", "Amsterdam"]

In [37]:
coords = { ciudad : get_geo_data(ciudad) for ciudad in ciudades}

/home/leandro/virtualenvs/DSS/lib/python3.4/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file /usr/lib/python3.4/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [41]:
mapa = folium.Map(location=[0, 0],zoom_start = 2)
color = rgb2hex([255,0,0])
for coord in coords:
    folium.Marker(coords[coord], popup=coord).add_to(mapa)
mapa.save('mapa.html')

In [42]:
%%HTML
<iframe width="100%" height="350" src="mapa.html?inline=true"></iframe>